<a href="https://colab.research.google.com/github/Zch6111/ML_Colab_CS329/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1 for CS 329P

**Authors**: YOUR_NAMES

**Emails**: YOUR_IDS@stanford.edu

**Submission.** Please insert your names and emails above, save your code in this notebook, and explain what you are doing along with your findings in text cells. You can think of it as a technical report with code. Before submission, please use `Kernel -> Restart & Run All` in the Jupyter menu to verify your code is runnable and save all outputs. Afterwards, you can either upload your raw notebook (`hw1.ipynb`) or an exported PDF version to the `Homework 1` assignment in Canvas.


In this homework, we will train a house sales price predictor on the data we scraped previously. The purpose of this homework is to let you practice different techniques that you can use to preprocess raw data. Your job is to obtain the best root mean squared logarithmic error (RMSLE) on the test dataset. To make your job easy, we provide sample code to train a model to report RMSLE and a list of ideas you can explore.

**Note**: You can use either local runtimes to complete this assignment, or a hosted runtime (with GPU) on Colab. The second option generally runs faster. If using a local runtime, make sure that your Python version is less than 3.9 but at least 3.6, or you may have issues installing Autogluon. If using a runtime hosted on Colab, you can use the File Explorer pane on the left to upload the `house_sales.ftr` file. Make sure to wait until the file finishes uploading before running the next code block.

Additionally, if using a local runtime, please refer to the [AG document](https://auto.gluon.ai/stable/index.html#installation) for info on how to install autogluon.

## Prepare Data

Let's first read in the dataset we used in our [Exploratory Data Analysis (EDA)](https://c.d2l.ai/stanford-cs329p/_static/notebooks/cs329p_notebook_eda.slides.html). Note that we use the [`feather` format](https://arrow.apache.org/docs/python/feather.html), which is faster to read than CSV but uses more disk space. The file `home_sales.ftr` can be downloaded from the Assignments folder in Canvas.

Just for your information, it is generated with:

```python
data = pd.read_csv('house_sales.zip', dtype='unicode')
data.to_feather('house_sales.ftr')
```

The following code needs at least 2GB memory. If using a local runtime, please make sure your machine has enough memory.

In [5]:
# Run the following line once to install. You may need to restart your runtime afterwards:
!pip3 install numpy pandas autogluon mxnet --upgrade
import pandas as pd
import numpy as np

data = pd.read_feather('house_sales.ftr')

  Using cached numpy-2.3.5-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)


In [8]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [10]:
import scipy
import numpy as np
scipy.__version__, np.__version__
print(data.columns)

Index(['Id', 'Address', 'Sold Price', 'Sold On', 'Summary', 'Type',
       'Year built', 'Heating', 'Cooling', 'Parking',
       ...
       'Well Disclosure', 'remodeled', 'DOH2', 'SerialX', 'Full Baths',
       'Tax Legal Lot Number', 'Tax Legal Block Number',
       'Tax Legal Tract Number', 'Building Name', 'Zip'],
      dtype='object', length=1789)


We select a few common columns to make our training fast. You need to select more columns to make your model more accurate.

In [13]:
df = data[['Sold Price', 'Sold On', 'Type', 'Year built', 'Bedrooms', 'Bathrooms' , 'Heating', 'Cooling', 'Parking', 'Zip']].copy()
# uncomment the below line to save memory
# del data
df.head()

,Sold Price,Sold On,Type,Year built,Bedrooms,Bathrooms,Heating,Cooling,Parking,Zip
0,"$2,000,000",01/31/20,SingleFamily,No Data,2,2.0,No Data,No Data,0 spaces,95046
1,"$2,100,000",02/25/21,SingleFamily,1951,3,3.0,Central,"Central Air, Dual","Driveway, Driveway - Brick",91011
2,"$1,125,000",11/06/19,SingleFamily,1954,3,2.0,Central Forced Air - Gas,Central AC,"Garage, Garage - Attached, Covered",95051
3,"$36,250,000",10/02/20,Apartment,1989,0,0.0,Other,No Data,"Mixed, Covered",95035
4,"$140,000",10/19/20,VacantLand,No Data,None,None,No Data,No Data,0 spaces,92561


We copy the code from EDA to convert `Sold Price` to numerical values, which is our prediction target. We also remove examples whose prices are too high or too low.

In [14]:
c = 'Sold Price'
if c in df.select_dtypes('object').columns:
    df.loc[:,c] = np.log10(
            pd.to_numeric(df[c].replace(r'[$,-]', '', regex=True)) + 1)
df = df[(df['Sold Price'] >= 4 ) & (df['Sold Price'] <= 8 )]

We use the house sales between 2021-2-15 and 2021-3-1 as our test data. You can use any example before 2021-2-15, but not after. In other words, we pretend we are launching our model on 2021-2-15 and testing it for 2 weeks. Here we only use sales in 2021 for fast training, but you can use more to improve accuracy.

In [23]:
test_start, test_end = pd.Timestamp(2021, 2, 15), pd.Timestamp(2021, 3, 1)
train_start = pd.Timestamp(2000, 1, 1)
df['Sold On'] = pd.to_datetime(df['Sold On'], errors='coerce')
train = df[(df['Sold On'] >= train_start) & (df['Sold On'] < test_start)]
test = df[(df['Sold On'] >= test_start) & (df['Sold On'] < test_end)]
train.shape, test.shape

((148183, 10), (11510, 10))

Define our evaluation metric.

In [24]:
def rmsle(y_hat, y):
    # we already used log prices before, so we only need to compute RMSE
    return sum((y_hat - y)**2 / len(y))**0.5

## AutoGluon Baseline

We provide a baseline model trained by AutoGluon (AG). AG is an automl tool that performs automatic feature engineering, model selections, and ensemble. You are welcome to use any model and tool in achieving the best results possible in your homework. However, we recommend that you reuse the following training code so that you can focus on data preprocessing.

In [26]:
from autogluon.tabular import TabularPredictor

label = 'Sold Price'
predictor = TabularPredictor(label=label, eval_metric='root_mean_squared_error').fit(train,presets='best_quality',time_limit=1200)

No path specified. Models will be saved in: "AutogluonModels/ag-20251123_074612"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       4.55 GB / 12.67 GB (35.9%)
Disk Space Avail:   62.40 GB / 107.72 GB (57.9%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
/usr/local/lib/python3.12/dist-packages/autogluon/core/utils/utils.py:701: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace([np.inf, -np.inf], np.nan, inplace=False)
Setting dynamic_stacking from 'auto' to True. Reason

Test the performance of each model.

In [27]:
predictor.leaderboard(test, silent=True)

/usr/local/lib/python3.12/dist-packages/autogluon/tabular/learner/abstract_learner.py:551: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_internal = y_internal.fillna(-1)


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L2,-0.202475,-0.249790,root_mean_squared_error,40.742207,111.952095,744.252663,6.971152,21.356692,196.470196,2,True,4
1,WeightedEnsemble_L3,-0.202694,-0.249597,root_mean_squared_error,40.746083,111.954637,744.339525,0.003876,0.002542,0.086862,3,True,5
2,LightGBMXT_BAG_L1,-0.205734,-0.253623,root_mean_squared_error,33.286893,89.151297,480.592048,33.286893,89.151297,480.592048,1,True,1
3,WeightedEnsemble_L2,-0.205734,-0.253623,root_mean_squared_error,33.289962,89.157175,480.673162,0.003069,0.005878,0.081114,2,True,3
4,LightGBM_BAG_L1,-0.403893,-0.430704,root_mean_squared_error,0.484162,1.444107,67.190419,0.484162,1.444107,67.190419,1,True,2


Next, we compute the importance of each feature, along with several other metrics. It loooks like the `Sold On` feature is not very useful, likely because the houses in the test data were all sold late. You can choose to either remove such a feature, or find a way to extract a more useful presentation from it.

In [28]:
predictor.feature_importance(test)

Computing feature importance via permutation shuffling for 9 features using 5000 rows with 5 shuffle sets...
	900.81s	= Expected runtime (180.16s per shuffle set)
	892.32s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Zip,0.091781,0.005129,1.165361e-06,5,0.102341,0.081220
Bathrooms,0.058704,0.001708,8.590014e-08,5,0.062221,0.055187
Type,0.054656,0.003355,1.695082e-06,5,0.061564,0.047748
Parking,0.040184,0.002093,8.794715e-07,5,0.044493,0.035875
Year built,0.020989,0.002088,1.160745e-05,5,0.025289,0.016689
Cooling,0.017305,0.001206,2.809403e-06,5,0.019788,0.014823
Bedrooms,0.014418,0.002219,6.523937e-05,5,0.018987,0.009849
Heating,0.010660,0.000988,8.745426e-06,5,0.012694,0.008627
Sold On,0.000102,0.000478,3.297569e-01,5,0.001086,-0.000883


Finally, let's predict and evaluate the RMSLE.

In [29]:
preds = predictor.predict(test.drop(columns=[label]))
rmsle(preds, test[label])

0.20269388027274635

## Your Solution

Please include your solution in the following section. (You are welcome to edit and delete code in previous sections).

Your goal is to train a model using the features in the original dataset that minimizes the RMSLE on the validation dataset. While the naïve model achieves an RMSLE of ~0.3, it is possible to achieve an RMSLE of less than 0.08 on the same dataset.

Here is a list of ideas you could explore:

- More features: We only selected a small set of columns to use in training. You can add more, especially the ones we examined in EDA.
- Data type conversion: Most data columns are strings; you may need to convert them into numerical values.
- Data cleaning: There are NAN and outliers sprinkled throughout the dataset. You should find ways to selectively filter and remove them.
- More examples: We only included sales made in 2021; there is a large number of examples in previous years that you can also include.

In [ ]:
# YOUR SOLUTION HERE

FIN